In [1]:
import json
import boto3
import asyncio
import argparse
from boto3.session import Session

In [2]:
async def invoke_agent(question):
    boto_session = Session()
    region_name = boto3.Session().region_name


    #SSM을 활용해서 Bedrock AgenticCore ARN 가져오기
    ssm_client = boto_session.client('ssm', region_name=region_name)
    agent_arn_response = ssm_client.get_parameter(
        Name="/mcp_agentic_core/runtime_iam/agent_arn"
    )
    agentcore_client = boto3.client('bedrock-agentcore', region_name=region_name)

    
    #Streaming으로 Bedrock AgenticCore Application 호출하기
    loop = asyncio.get_event_loop()
    response = await loop.run_in_executor(
        None,
        lambda: agentcore_client.invoke_agent_runtime(
            agentRuntimeArn=agent_arn_response["Parameter"]["Value"],
            qualifier="DEFAULT",
            payload=json.dumps({"input_data": question})
        )
    )

    #응답을 객체로 받은뒤 계속적으로 오는 streaming 결과 출력
    for line in response['response'].iter_lines():
        if line and line.startswith(b'data: '):
            data = json.loads(line[6:].decode('utf-8'))
            if data['type'] == 'stream':
                print(data['content'], end='')
    print()

In [3]:
await invoke_agent("3*3에 대해 알려주세요")


RuntimeClientError: An error occurred (RuntimeClientError) when calling the InvokeAgentRuntime operation: An error occurred when starting the runtime. Please check your CloudWatch logs for more information.